# Image Classification using TensorFlow v2

## The problem

To do image classification to identify objects like airplane, automobile, bird, cat, deer, dog, frog, horse, ship and trunk. We shall use Deep Learning framework TensorFlow to train a CNN model backed by [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html). 

CIFAR is an acronym that stands for the [Canadian Institute For Advanced Research](https://cifar.ca/) and the [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) was developed along with the CIFAR-100 dataset by researhers at the CIFAR instite.

For [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html), it consists of 60,000 32x32 pixel color pictures of objects from 10 classes, such as bird, cat and deer, etc. The class labels and their standard associated integer values are listed below:

* 0: airplane
* 1: automobile
* 2: bird
* 3: cat
* 4: deer
* 5: dog
* 6: frog
* 7: horse
* 8: ship
* 9: truck



## Setup

To import packages.

In [1]:
import sys
import IPython

!{sys.executable} -m pip install --upgrade pip --user
!{sys.executable} -m pip install matplotlib tensorflow-datasets ipywidgets opencv-python --user
IPython.Application.instance().kernel.do_shutdown(True)

  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


{'status': 'ok', 'restart': True}

In [2]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds

### Download the dataset

In [3]:
(ds_train, ds_test), info = tfds.load('cifar10', split=['train', 'test'], as_supervised=True, with_info=True, shuffle_files=True)

In [4]:
info

tfds.core.DatasetInfo(
    name='cifar10',
    full_name='cifar10/3.0.2',
    description="""
    The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.
    """,
    homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
    data_path='/root/tensorflow_datasets/cifar10/3.0.2',
    download_size=162.17 MiB,
    dataset_size=132.40 MiB,
    features=FeaturesDict({
        'id': Text(shape=(), dtype=tf.string),
        'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=50000, num_shards=1>,
    },
    citation="""@TECHREPORT{Krizhevsky09learningmultiple,
        author = {Alex Krizhevsky},
        title = {Learning multiple laye

In [5]:
class_names = info.features['label'].names
class_names

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [ ]:
!ls -lstr ~/tensorflow_datasets/cifar10/3.0.2/cifar10-train.tfrecord-00000-of-00001

In [ ]:
!ls -l /root/tensorflow_datasets/cifar10/3.0.2/


In [ ]:
!ls -l $filenames

In [ ]:
from functools import partial

BATCH_SIZE = 64
IMAGE_SIZE = [32, 32]


def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image


def read_tfrecord(example, labeled):
    tfrecord_format = (
        {
            "image": tf.io.FixedLenFeature([], tf.string),
            "label": tf.io.FixedLenFeature([], tf.int64),
        }
        if labeled
        else {"image": tf.io.FixedLenFeature([], tf.string),}
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example["image"])
    if labeled:
        label = tf.cast(example["label"], tf.int32)
        return image, label
    return image


def load_dataset(filenames, labeled=True):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(
        filenames
    )  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord, labeled=labeled), num_parallel_calls=BATCH_SIZE
    )
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames, labeled=labeled)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=BATCH_SIZE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset


In [ ]:
filename = "/root/tensorflow_datasets/cifar10/3.0.2/cifar10-train.tfrecord-00000-of-00001"
dataset = get_dataset([filename])

In [ ]:
image_batch, label_batch = next(iter(dataset))


def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10, 10))
    for n in range(25):
        ax = plt.subplot(5, 5, n + 1)
        plt.imshow(image_batch[n] / 255.0)
        title = class_names[label_batch[n].item()]
        plt.title(title)
        plt.axis("off")


show_batch(image_batch.numpy(), label_batch.numpy())


In [ ]:
label_batch.numpy()

In [ ]:
label_batch.numpy()[1].item(), class_names

In [ ]:
fig = tfds.show_examples(ds_train, info)

#### To build the training data pipeline

In [6]:
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(normalize_img)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(10)

#### To build the validation data pipeline

In [7]:
ds_test = ds_test.map(normalize_img)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(10)

### Create and train the model

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (32, 32, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(len(class_names), activation = 'softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
model.summary()

[2022-01-28 22:21:38.477 tensorflow-2-3-cpu-py-ml-t3-medium-37f9e9ca00776b5a267026dff80d:376 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-01-28 22:21:38.822 tensorflow-2-3-cpu-py-ml-t3-medium-37f9e9ca00776b5a267026dff80d:376 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 32)          0         
_____________________

In [9]:
model.fit(ds_train, epochs=6, validation_data=ds_test)

Epoch 1/6
391/391 [==============================] - 19s 48ms/step - loss: 2.1198 - sparse_categorical_accuracy: 0.3371 - val_loss: 2.0464 - val_sparse_categorical_accuracy: 0.4102
Epoch 2/6
391/391 [==============================] - 17s 43ms/step - loss: 2.0174 - sparse_categorical_accuracy: 0.4417 - val_loss: 1.9975 - val_sparse_categorical_accuracy: 0.4590
Epoch 3/6
391/391 [==============================] - 16s 41ms/step - loss: 1.9680 - sparse_categorical_accuracy: 0.4924 - val_loss: 1.9625 - val_sparse_categorical_accuracy: 0.4968
Epoch 4/6
391/391 [==============================] - 16s 41ms/step - loss: 1.9343 - sparse_categorical_accuracy: 0.5283 - val_loss: 1.9230 - val_sparse_categorical_accuracy: 0.5375
Epoch 5/6
391/391 [==============================] - 16s 42ms/step - loss: 1.9114 - sparse_categorical_accuracy: 0.5503 - val_loss: 1.9220 - val_sparse_categorical_accuracy: 0.5379
Epoch 6/6
391/391 [==============================] - 17s 43ms/step - loss: 1.8966 - sparse_cate

In [10]:
scores = model.evaluate(ds_test, verbose=2)

79/79 - 1s - loss: 1.8916 - sparse_categorical_accuracy: 0.5682


In [11]:
print(
        "Validation results: "
        + "; ".join(map(
            lambda i: f"{model.metrics_names[i]}={scores[i]:.5f}", range(len(model.metrics_names))
        ))
    )


Validation results: loss=1.89163; sparse_categorical_accuracy=0.56820


In [12]:
image = cv2.imread("data/cat.png", 1)

# resize, as our model is expecting images in 32x32.
image = cv2.resize(image, (32, 32))
image = image / 255.0

image = np.expand_dims(image, axis = 0)

In [13]:
for img, label in ds_test.take(1):
    labels = label
    images = img

In [39]:
index = 4
# target_image = np.expand_dims(images[index], axis = 0)
target_image = image
pred = model.predict(target_image)
print(pred, np.argmax(pred))

print(f"prediction: {class_names[np.argmax(pred)]}")
print(f"actual: {class_names[labels[index]]}")

[[2.2802919e-01 6.5504567e-04 7.2071500e-02 1.1517079e-02 2.9452860e-03
  1.0040117e-03 2.4559739e-07 1.7016700e-06 6.8375224e-01 2.3688415e-05]] 8
prediction: ship
actual: dog


In [36]:
export_path = './model/1'

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None

)

INFO:tensorflow:Assets written to: ./model/1/assets


INFO:tensorflow:Assets written to: ./model/1/assets


In [ ]:
!apt-get install gnupg

In [24]:
%%sh

echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -

OSError: Background processes not supported.

In [22]:
!apt-get update && apt-get install tensorflow-model-server



Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]               
Get:3 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [26.8 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2517 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]    
Get:7 http://archive.ubuntu.com/ubuntu bionic/restricted amd64 Packages [13.5 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic/multiverse amd64 Packages [186 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic/main amd64 Packages [1344 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [738 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic/universe amd64 Packages [11.3 MB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1463 kB]
Get:13 ht

In [ ]:
!tensorflow_model_server --rest_api_port=8501 --model_name=cifar10-model --model_base_path=/root/amazon-sagemaker-workshop-n/cv_byo/model > server.log 2>$1


In [ ]:
def show(idx, title):
  plt.figure()
  plt.imshow(test_images[idx].reshape(28,28))
  plt.axis('off')
  plt.title('\n\n{}'.format(title), fontdict={'size': 16})

import random
rando = random.randint(0,len(test_images)-1)
show(rando, 'An Example Image: {}'.format(class_names[test_labels[rando]]))


In [29]:
import json
image = cv2.imread("data/cat.png", 1)

# resize, as our model is expecting images in 32x32.
image = cv2.resize(image, (32, 32))
image = image / 255.0

image = np.expand_dims(image, axis = 0)



data = json.dumps({
    "signature_name": "serving_default", "instances": image[:].tolist()
})

# print(data)

In [30]:
!pip install -q requests

In [37]:


import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/cifar10-model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']
print(predictions)
print(np.argmax(predictions))


[[0.228029341, 0.000655044627, 0.0720714778, 0.0115170609, 0.00294528017, 0.00100401102, 2.45596397e-07, 1.701668e-06, 0.68375212, 2.36883661e-05]]
8


In [ ]:
# training part
%%sh

python ./container/cifar10/cifar10.py --data-dir /root/data/ --model-dir ./model --train-steps 5

In [ ]:
data_location = "s3://sagemaker-ap-southeast-2-835880313890/DEMO-tensorflow-cifar10/"



### Dataset

We will be using Keras (CNN) to train a computer vision model backed by [Intel Image Scene Classification of Multiclass](https://www.kaggle.com/puneet6060/intel-image-classification) image dataset.

### Download Intel Image Classification dataset

Run below shell command to download and unzip the images
```shell
# please use the proper folder name on your local
folder=amazon-sagemaker-workshop-n

cd ~/$folder/02_bring_your_own_container/data

time wget https://df4l9poikws9t.cloudfront.net/shared/sagemaker/data/intel_image_classification.zip -O ./intel_image_classification.zip

time unzip intel_image_classification.zip 
```


### Intel Image Classification

This notebook will do basic analysis on the image data and train a simple image classification model using regression model.

### Setup

In [ ]:
import sys
import IPython
!{sys.executable} -m pip install seaborn tqdm
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from tqdm import tqdm

In [ ]:
class_names = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)

IMAGE_SIZE = (150, 150)

### Loading the data

In [ ]:
def load_data(datasets = ['./data/seg_train/seg_train', './data/seg_test/seg_test']):
    output = []
    
    for dataset in datasets:
        image_paths = []
        labels = []
        
        print(f"Loading images from {dataset}")
        
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                image_paths.append(img_path)
                labels.append(label)
                       
        output.append((image_paths, labels))
        
    return output

In [ ]:
%%time

(train_images, train_labels), (test_images, test_labels) = load_data()

In [ ]:
train_images, train_labels = shuffle(train_images, train_labels, random_state = 32)

In [ ]:
print(f"training dataset amount: {len(train_labels)}")
print(f"testing dataset amount: {len(test_labels)}")
print(f"image size: {IMAGE_SIZE}")

In [ ]:
import pandas as pd

_, train_counts = np.unique(train_labels, return_counts = True)
_, test_counts = np.unique(test_labels, return_counts = True)

pd.DataFrame({'train':train_counts, 'test': test_counts}, index = class_names).plot.bar()
plt.show()

In [ ]:
plt.pie(
    train_counts, 
    explode = (0, 0, 0, 0, 0, 0),
    labels = class_names,
    autopct = '%1.1f%%')
    
plt.axis('equal')
plt.title('Proportion of each observed category - Train Dataset')
plt.show()

In [ ]:
plt.pie(
    test_counts, 
    explode = (0, 0, 0, 0, 0, 0),
    labels = class_names,
    autopct = '%1.1f%%')
    
plt.axis('equal')
plt.title('Proportion of each observed category - Test Dataset')
plt.show()

#### Visualize images

In [ ]:
from PIL import Image
def get_image_array(image_path):
    img = Image.open(image_path, 'r')
    return np.array(img)

def display_examples(class_names, image_paths, labels):
    fig = plt.figure(figsize = (10, 10))
    fig.suptitle('Some examples of images of the dataset', fontsize = 16)
    
    for i in range(20):
        plt.subplot(4, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(get_image_array(image_paths[i]))
        plt.xlabel(class_names[labels[i]])
    plt.show()

In [ ]:
display_examples(class_names, train_images, train_labels)

#### Data Augmentation
Image augmentation applies transforms to an image and results in additional images that the network can train on. Image data generator has many options and also allows custom preprocessing functions through the parameter of the same name.

In [ ]:
tf.random.set_seed(99)

def blur_preprocessing(img):
    return cv2.blur(img, (5, 5))

# training data generator
train_data_generator = ImageDataGenerator(
    rescale = 1.0 / 255.0, 
    validation_split = 0.0, # it's for training dataset only
    rotation_range = 180,
    horizontal_flip = True,
    vertical_flip = True,
    preprocessing_function = blur_preprocessing
)

# training dataset
train_data_multi = train_data_generator.flow_from_directory(
    directory = './data/seg_train/seg_train',
    target_size = IMAGE_SIZE,
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed = 42
)

# testing data generator
validation_data_generator = ImageDataGenerator(
    rescale = 1.0 / 255.0
)

# testing dataset
validation_data_multi = validation_data_generator.flow_from_directory(
    directory = './data/seg_test/seg_test',
    target_size = IMAGE_SIZE,
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed = 42
)

### Model Training Using CNN

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(len(class_names), activation = 'softmax')
])

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.summary()

In [ ]:
%%time 

history = model.fit(
    train_data_multi, 
    batch_size = 128,
    epochs = 5,
    validation_data = validation_data_multi,
    verbose = 2
)